### This script allows to get better control over tasks in earth engine

* Purpose of script: allows the user to list tasks and cancel all pending tasks
* Author: Rutger Hofste
* Kernel used: python27
* Date created: 20170913

In [1]:
import time
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
print(dateString,timeString)

Y2018M04D20 UTC 08:55


# Settings

In [2]:
MAXTASKS = 500
CANCELTASKS = 1 # Cancels all pending tasks

In [3]:
import pandas as pd
import ee
from retrying import retry
import datetime
import random

In [4]:
ee.Initialize()

# Functions

In [9]:
def get_tasks():
    return ee.batch.Task.list()

def cancel_task(task):
    print(task)
    random_time = random.random()
    time.sleep(0.5+random_time*0.5)
    if task.config['state'] in (u'RUNNING',u'UNSUBMITTED',u'READY') :
        print('canceling %s' % task)
        task.cancel()
        
        
@retry(wait_exponential_multiplier=10000, wait_exponential_max=100000)
def checkStatus(task):
    return ee.batch.Task.status(task)
           
def get_details(taskList,MAXTASKS):
    df = pd.DataFrame()
    for i in range(0,min(len(taskList),MAXTASKS)):
        dictNew = checkStatus(taskList[i])
        dfNew = pd.DataFrame(dictNew, index=[i])
        try:
            dfNew["calctime(min)"] = (dfNew["update_timestamp_ms"]-dfNew["start_timestamp_ms"])/(1000*60)
            dfNew["queuetime(min)"] = (dfNew["start_timestamp_ms"]-dfNew["creation_timestamp_ms"])/(1000*60)
            dfNew["runtime(min)"]= dfNew["queuetime(min)"]+dfNew["calctime(min)"]
            dfNew["start_timestamp_UTC"] = datetime.datetime.fromtimestamp(dfNew["start_timestamp_ms"]/1000).strftime('%H:%M:%S')
        except:
            pass
        df = df.append(dfNew)
        print(i)
    return df
    


In [10]:
taskList = get_tasks()


In [11]:
type(taskList)

list

In [13]:
len(taskList)

14207

In [ ]:
detailedTasks = get_details(taskList,MAXTASKS)

In [ ]:
detailedTasks

# DANGER ZONE

In [12]:
if CANCELTASKS == 1:
    pendingTasks = [task for task in taskList if task.config['state'] in (u'RUNNING',u'UNSUBMITTED',u'READY')]
    for task in pendingTasks:
        cancel_task(task)

<Task EXPORT_IMAGE: global_historical_PDomWW_month_m_5min_1960_2014_I125Y1970M06_V02 (READY)>
canceling <Task EXPORT_IMAGE: global_historical_PDomWW_month_m_5min_1960_2014_I125Y1970M06_V02 (READY)>
<Task EXPORT_IMAGE: global_historical_PDomWW_month_m_5min_1960_2014_I124Y1970M05_V02 (READY)>
canceling <Task EXPORT_IMAGE: global_historical_PDomWW_month_m_5min_1960_2014_I124Y1970M05_V02 (READY)>
<Task EXPORT_IMAGE: global_historical_PDomWW_month_m_5min_1960_2014_I123Y1970M04_V02 (READY)>
canceling <Task EXPORT_IMAGE: global_historical_PDomWW_month_m_5min_1960_2014_I123Y1970M04_V02 (READY)>
<Task EXPORT_IMAGE: global_historical_PDomWW_month_m_5min_1960_2014_I122Y1970M03_V02 (READY)>
canceling <Task EXPORT_IMAGE: global_historical_PDomWW_month_m_5min_1960_2014_I122Y1970M03_V02 (READY)>
<Task EXPORT_IMAGE: global_historical_PDomWW_month_m_5min_1960_2014_I121Y1970M02_V02 (READY)>
canceling <Task EXPORT_IMAGE: global_historical_PDomWW_month_m_5min_1960_2014_I121Y1970M02_V02 (READY)>
<Task EXPO

canceling <Task EXPORT_IMAGE: global_historical_PDomWW_month_m_5min_1960_2014_I084Y1967M01_V02 (READY)>
<Task EXPORT_IMAGE: global_historical_PDomWW_month_m_5min_1960_2014_I083Y1966M12_V02 (READY)>
canceling <Task EXPORT_IMAGE: global_historical_PDomWW_month_m_5min_1960_2014_I083Y1966M12_V02 (READY)>
<Task EXPORT_IMAGE: global_historical_PDomWW_month_m_5min_1960_2014_I082Y1966M11_V02 (READY)>
canceling <Task EXPORT_IMAGE: global_historical_PDomWW_month_m_5min_1960_2014_I082Y1966M11_V02 (READY)>
<Task EXPORT_IMAGE: global_historical_PDomWW_month_m_5min_1960_2014_I081Y1966M10_V02 (READY)>
canceling <Task EXPORT_IMAGE: global_historical_PDomWW_month_m_5min_1960_2014_I081Y1966M10_V02 (READY)>
<Task EXPORT_IMAGE: global_historical_PDomWW_month_m_5min_1960_2014_I080Y1966M09_V02 (READY)>
canceling <Task EXPORT_IMAGE: global_historical_PDomWW_month_m_5min_1960_2014_I080Y1966M09_V02 (READY)>
<Task EXPORT_IMAGE: global_historical_PDomWW_month_m_5min_1960_2014_I079Y1966M08_V02 (READY)>
canceling 

<Task EXPORT_IMAGE: global_historical_PIrrWW_month_m_5min_1960_2014_I131Y1970M12_V02 (READY)>
canceling <Task EXPORT_IMAGE: global_historical_PIrrWW_month_m_5min_1960_2014_I131Y1970M12_V02 (READY)>
<Task EXPORT_IMAGE: global_historical_PIrrWW_month_m_5min_1960_2014_I130Y1970M11_V02 (READY)>
canceling <Task EXPORT_IMAGE: global_historical_PIrrWW_month_m_5min_1960_2014_I130Y1970M11_V02 (READY)>
<Task EXPORT_IMAGE: global_historical_PIrrWW_month_m_5min_1960_2014_I129Y1970M10_V02 (READY)>
canceling <Task EXPORT_IMAGE: global_historical_PIrrWW_month_m_5min_1960_2014_I129Y1970M10_V02 (READY)>
<Task EXPORT_IMAGE: global_historical_PIrrWW_month_m_5min_1960_2014_I128Y1970M09_V02 (READY)>
canceling <Task EXPORT_IMAGE: global_historical_PIrrWW_month_m_5min_1960_2014_I128Y1970M09_V02 (READY)>
<Task EXPORT_IMAGE: global_historical_PIrrWW_month_m_5min_1960_2014_I127Y1970M08_V02 (READY)>
canceling <Task EXPORT_IMAGE: global_historical_PIrrWW_month_m_5min_1960_2014_I127Y1970M08_V02 (READY)>
<Task EXPO

canceling <Task EXPORT_IMAGE: global_historical_PIrrWW_month_m_5min_1960_2014_I090Y1967M07_V02 (READY)>
<Task EXPORT_IMAGE: global_historical_PIrrWW_month_m_5min_1960_2014_I089Y1967M06_V02 (READY)>
canceling <Task EXPORT_IMAGE: global_historical_PIrrWW_month_m_5min_1960_2014_I089Y1967M06_V02 (READY)>
<Task EXPORT_IMAGE: global_historical_PIrrWW_month_m_5min_1960_2014_I088Y1967M05_V02 (READY)>
canceling <Task EXPORT_IMAGE: global_historical_PIrrWW_month_m_5min_1960_2014_I088Y1967M05_V02 (READY)>
<Task EXPORT_IMAGE: global_historical_PIrrWW_month_m_5min_1960_2014_I087Y1967M04_V02 (READY)>
canceling <Task EXPORT_IMAGE: global_historical_PIrrWW_month_m_5min_1960_2014_I087Y1967M04_V02 (READY)>
<Task EXPORT_IMAGE: global_historical_PIrrWW_month_m_5min_1960_2014_I086Y1967M03_V02 (READY)>
canceling <Task EXPORT_IMAGE: global_historical_PIrrWW_month_m_5min_1960_2014_I086Y1967M03_V02 (READY)>
<Task EXPORT_IMAGE: global_historical_PIrrWW_month_m_5min_1960_2014_I085Y1967M02_V02 (READY)>
canceling 

<Task EXPORT_IMAGE: global_historical_PIrrWW_month_m_5min_1960_2014_I048Y1964M01_V02 (READY)>
canceling <Task EXPORT_IMAGE: global_historical_PIrrWW_month_m_5min_1960_2014_I048Y1964M01_V02 (READY)>
<Task EXPORT_IMAGE: global_historical_PIrrWW_month_m_5min_1960_2014_I047Y1963M12_V02 (READY)>
canceling <Task EXPORT_IMAGE: global_historical_PIrrWW_month_m_5min_1960_2014_I047Y1963M12_V02 (READY)>
<Task EXPORT_IMAGE: global_historical_PIrrWW_month_m_5min_1960_2014_I046Y1963M11_V02 (READY)>
canceling <Task EXPORT_IMAGE: global_historical_PIrrWW_month_m_5min_1960_2014_I046Y1963M11_V02 (READY)>
<Task EXPORT_IMAGE: global_historical_PIrrWW_month_m_5min_1960_2014_I045Y1963M10_V02 (READY)>
canceling <Task EXPORT_IMAGE: global_historical_PIrrWW_month_m_5min_1960_2014_I045Y1963M10_V02 (READY)>
<Task EXPORT_IMAGE: global_historical_PIrrWW_month_m_5min_1960_2014_I044Y1963M09_V02 (READY)>
canceling <Task EXPORT_IMAGE: global_historical_PIrrWW_month_m_5min_1960_2014_I044Y1963M09_V02 (READY)>
<Task EXPO

...

canceling <Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I489Y2000M10_V02 (READY)>
<Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I488Y2000M09_V02 (READY)>
canceling <Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I488Y2000M09_V02 (READY)>
<Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I487Y2000M08_V02 (READY)>
canceling <Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I487Y2000M08_V02 (READY)>
<Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I486Y2000M07_V02 (READY)>
canceling <Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I486Y2000M07_V02 (READY)>
<Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I485Y2000M06_V02 (READY)>
canceling <Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I485Y2000M06_V02 (READY)>
<Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I484Y2000M05_V02 (READY)>
canceling 

<Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I447Y1997M04_V02 (READY)>
canceling <Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I447Y1997M04_V02 (READY)>
<Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I446Y1997M03_V02 (READY)>
canceling <Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I446Y1997M03_V02 (READY)>
<Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I445Y1997M02_V02 (READY)>
canceling <Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I445Y1997M02_V02 (READY)>
<Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I444Y1997M01_V02 (READY)>
canceling <Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I444Y1997M01_V02 (READY)>
<Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I443Y1996M12_V02 (READY)>
canceling <Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I443Y1996M12_V02 (READY)>
<Task EXPO

canceling <Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I406Y1993M11_V02 (READY)>
<Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I405Y1993M10_V02 (READY)>
canceling <Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I405Y1993M10_V02 (READY)>
<Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I404Y1993M09_V02 (READY)>
canceling <Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I404Y1993M09_V02 (READY)>
<Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I403Y1993M08_V02 (READY)>
canceling <Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I403Y1993M08_V02 (READY)>
<Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I402Y1993M07_V02 (READY)>
canceling <Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I402Y1993M07_V02 (READY)>
<Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I401Y1993M06_V02 (READY)>
canceling 

<Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I364Y1990M05_V02 (READY)>
canceling <Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I364Y1990M05_V02 (READY)>
<Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I363Y1990M04_V02 (READY)>
canceling <Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I363Y1990M04_V02 (READY)>
<Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I362Y1990M03_V02 (READY)>
canceling <Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I362Y1990M03_V02 (READY)>
<Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I361Y1990M02_V02 (READY)>
canceling <Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I361Y1990M02_V02 (READY)>
<Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I360Y1990M01_V02 (READY)>
canceling <Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I360Y1990M01_V02 (READY)>
<Task EXPO

canceling <Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I323Y1986M12_V02 (READY)>
<Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I322Y1986M11_V02 (READY)>
canceling <Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I322Y1986M11_V02 (READY)>
<Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I321Y1986M10_V02 (READY)>
canceling <Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I321Y1986M10_V02 (READY)>
<Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I320Y1986M09_V02 (READY)>
canceling <Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I320Y1986M09_V02 (READY)>
<Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I319Y1986M08_V02 (READY)>
canceling <Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I319Y1986M08_V02 (READY)>
<Task EXPORT_IMAGE: global_historical_PDomWN_month_m_5min_1960_2014_I318Y1986M07_V02 (READY)>
canceling 